In [3]:
# sine part to find the angles:

import numpy as np
from scipy.special import jv
from scipy.optimize import minimize
from functools import reduce

# ----- Symbolic Polynomial Algebra -----

def symbolic_poly_multiply(p1, p2):
    result_len = len(p1) + len(p2) - 1
    result = [0j] * result_len
    for i in range(len(p1)):
        for j in range(len(p2)):
            result[i + j] += p1[i] * p2[j]
    return result


def symbolic_poly_multiply_offdiagonal(p1, p2):
    # Original off-diagonal multiplication (used for building R)
    result_len = len(p1) + len(p2) + 1
    result = [0j] * result_len
    for i in range(len(p1)):
        for j in range(len(p2)):
            term = p1[i] * p2[j]
            result[i + j + 1] += term
            result[i + j + 2] -= term
    return result


def add_symbolic_polynomials(p1, p2):
    max_len = max(len(p1), len(p2))
    p1 += [0j] * (max_len - len(p1))
    p2 += [0j] * (max_len - len(p2))
    return [a + b for a, b in zip(p1, p2)]


# ----- Symbolic Matrix Multiplication -----

def multiply_symbolic_matrices(A, B):
    R = np.empty((2, 2), dtype=object)
    for i in range(2):
        for j in range(2):
            result = [0j]
            for k in range(2):
                if i != k and j != k:
                    prod = symbolic_poly_multiply_offdiagonal(A[i, k], B[k, j])
                else:
                    prod = symbolic_poly_multiply(A[i, k], B[k, j])
                result = add_symbolic_polynomials(result, prod)
            R[i, j] = result
    return R


# ----- Create Numeric Matrix -----

def create_numeric_matrix(f, idx):
    A = np.empty((2, 2), dtype=object)

    # Start from f2 and f3 (not f1 and f2)
    f1, f2 = f[2 * idx + 1], f[2 * idx + 2]
    barf1 = np.conj(f1)

    a00 = barf1 * f2
    a01 = f1 * f2 - barf1 * f2
    a10 = a01
    a20 = -np.conj(a01)
    a30 = np.conj(a00)
    a31 = np.conj(a01)

    A[0, 0] = [a00, a01]
    A[0, 1] = [a10]
    A[1, 0] = [a20]
    A[1, 1] = [a30, a31]
    return A


# ----- Target Coefficients (odd Chebyshev) -----

# def target_coeffs(t, max_k, max_degree):
#     coeff_total = np.zeros(2 * max_degree)
#     T = np.zeros((2 * max_degree, 2 * max_degree))
#     T[0][0] = 1
#     T[1][1] = 1

#     # Build Chebyshev polynomials in power basis
#     for i in range(2, len(T)):
#         T[i, :] = 2 * np.concatenate((np.zeros(1), T[i - 1, 0:len(T[0]) - 1]))
#         T[i, :] -= T[i - 2, :]

#     # Use odd-order Chebyshev terms only: T_{2k+1}(x)
#     coeff_total = np.zeros_like(T[0, :])
#     for k in range(0, len(T) // 2 - 1):
#         coeff_total += 2 * (-1) ** k * jv(2 * k + 1, t) * T[2 * k + 1, :]

#     return coeff_total

def target_coeffs(t, max_k, max_degree):
    coeff_total = np.zeros(2 * max_degree)
    T = np.zeros((2 * max_degree, 2 * max_degree))
    T[0][0] = 1
    T[1][1] = 1
    for i in range(2, len(T)):
        T[i,:] = 2 * np.concatenate((np.zeros(1),T[i-1,0:len(T[0])-1]))
        T[i,:] -= T[i-2,:]
    coeff_total = 2 * jv(1, t) * T[1, :]
    for k in range(3, len(T), 2):
    #     n = 2 * k
    #     J_val = jv(n, t)
    #     factor = (-1) ** k * 2 * J_val

        # # Get Chebyshev T_{2k}(x), convert to power basis
        # Tn = Chebyshev.basis(n)
        # coeffs = Tn.convert(kind=np.polynomial.Polynomial).coef
        # padded = np.pad(coeffs, (0, max_degree + 1 - len(coeffs)))

        # coeff_total += factor * padded
        coeff_total += 2 * (-1)**int((k-1)/2) * jv(k, t) * T[k, :] 
    return coeff_total

# ----- Cost Function -----

def compute_cost(phases, t):
    f = [np.exp(2j * phi) for phi in phases]

    # Create symbolic matrices (starting from f2, f3,...)
    matrices = [create_numeric_matrix(f, i) for i in range(len(f)//2)]

    # Multiply all matrices
    R = reduce(multiply_symbolic_matrices, matrices)

    R00 = R[0, 0]
    R10 = R[1, 0]

    # f1 = exp(2j * phi1)
    f1 = np.exp(2j * phases[0])

    # Diagonal multiply: f1 * R[0,0]
    diag_part = symbolic_poly_multiply([f1], R00)

    # Off-diagonal multiply (no shifting)
    r = R10
    offdiag_noshift = [f1 * r[0]]  # start with f1*r0
    for i in range(1, len(r)):
        if i < len(r) - 1:
            offdiag_noshift.append(f1 * (r[i] - r[i - 1]))
        else:
            offdiag_noshift.append(-f1 * r[i - 1])

    # Combine both parts
    Rprime00 = add_symbolic_polynomials(diag_part, offdiag_noshift)

    # Target coefficients
    target = target_coeffs(t, len(Rprime00) - 1, (len(Rprime00) - 1) * 2)
    target = target[1::2]

    # Cost function
    cost = 0.0
    for i in range(min(len(Rprime00), len(target))):
        cost += (Rprime00[i].real - target[i]) ** 2

    return cost


# ----- Run Optimization ---


if __name__ == "__main__":
    t = 2.5
    num_phases = 7

    initial_phases = np.random.rand(num_phases)

    result = minimize(compute_cost, initial_phases, args=(t,), method='BFGS')

    print("Optimized phases:")
    print(result.x)
    print("\nMinimum cost:")
    print(result.fun)





Optimized phases:
[ 0.47750707 -0.43960706  0.48742539  0.64623975  1.15423626 -0.00571177
  0.35075195]

Minimum cost:
3.057356153176736e-11
